In [4]:
import numpy as np
import pandas as pd

from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, AveragePooling1D, AveragePooling2D,TimeDistributed
from keras.layers import LSTM, SpatialDropout1D, Merge
from keras import backend
np.random.seed(0)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#load data
train = pd.read_csv('data/features/features.csv', sep=',', header=0)

#y labels
useful = train["useful"].values

#features
abstracts = train['fulltitle']+train['abstract']
abstracts = abstracts.values.astype(dtype=str)
#title = train['fulltitle'].values.astype(dtype=str)
journal = train['subtitle'].values.astype(dtype=str)

#only rows with abstract
idxthere = np.nonzero(train['abstract'] != ' ')[0]
#title = title[idxthere]
journal = journal[idxthere]
abstracts = abstracts[idxthere]
useful = useful[idxthere]

num_labels = len(np.unique(useful))

#text pre-processing
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
stop_words.remove('but')
stop_words.remove('not')
stop_words.remove('no')
stemmer = SnowballStemmer('english')

print("pre-processing train docs...")
processed_docs_train = []
for doc in abstracts:
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in filtered]
    processed_docs_train.append(stemmed)

dictionary = corpora.Dictionary(processed_docs_train)
dictionary_size = len(dictionary.keys())
print("dictionary size: ", dictionary_size)
#dictionary.save('dictionary.dict')
#corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

print("converting to token ids...")
word_id_train, word_id_len = [], []
for doc in processed_docs_train:
    word_ids = [dictionary.token2id[word] for word in doc]
    word_id_train.append(word_ids)
    word_id_len.append(len(word_ids))

seq_len = np.round((np.mean(word_id_len) + 2*np.std(word_id_len))).astype(int)

#pad sequences
word_id_train = sequence.pad_sequences(np.array(word_id_train), maxlen=seq_len)
#y_train_enc = np_utils.to_categorical(sentiment_train, num_labels)
#y_train_enc = (sentiment_train-2)/2 #map down to: [-1,1]

NameError: name 'title' is not defined

In [ ]:

#LSTM (using text from abstract+title)
branch1 = Sequential()
branch1.add(Embedding(dictionary_size, 128))
branch1.add(SpatialDropout1D(0.2))
branch1.add(LSTM(128, recurrent_dropout=0.2, dropout=0.2))
#add non-text input

#second branch: using journal
branch2 = Sequential()
branch2.add(Embedding(n_journals, 32))
branch2.add(Dense(32))

model = Sequential()
model.add(Merge([branch1, branch2]), mode="concat")
#try a pooling inbetween
#model.add(TemporalMeanPooling())
model.add(Dense(64), activation = "relu")
model.add(Dense(1))
model.add(Activation('tanh'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','binary_accuracy'])


In [12]:
random_idx = np.arange(word_id_train.shape[0])
np.random.shuffle(random_idx)
word_id_train = word_id_train[random_idx,:]
useful = useful[random_idx]

In [13]:
from keras.callbacks import EarlyStopping
print("fitting LSTM ...")
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit([word_id_train, journals], useful, epochs=1, batch_size=64, verbose=1, validation_split=0.2, callbacks=[early_stopping])
test_pred = model.predict_classes(word_id_test)

fitting LSTM ...
Train on 36708 samples, validate on 9177 samples
Epoch 1/1
25600/36708 [===================>..........] - ETA: 330s - loss: 0.2215 - mean_absolute_error: 0.4300 

KeyboardInterrupt: 

In [54]:
test_pred = model.predict(word_id_test)
test_pred_conv = np.around((test_pred*2)+2)

In [63]:
#make a submission
test_df['Sentiment'] = test_pred_conv.astype(int)
header = ['PhraseId', 'Sentiment']
test_df.to_csv('./lstm_sentiment3.csv', columns=header, index=False, header=True)